In [57]:
import pandas as pd
import numpy as np
import json
import ast
import os

In [58]:
from tqdm.notebook import tqdm

In [59]:
tqdm.pandas()

In [60]:

args = pd.Series({
    "root_dir":"/mnt/disks/data/",
    "dataset_path":"/mnt/disks/data/fma/fma_large",
    "embeddings":"music_style",
    "train_id": "hierarchical_all",
    'sample_size': 1
})


In [61]:

job_path = "/mnt/disks/data/fma/trains"

# In[15]:

train_path = os.path.join(job_path,args.train_id)

# In[16]:

base_path = os.path.join(args.root_dir,"fma")

# In[17]:

models_path = os.path.join(args.root_dir,"models")


metadata_path_fma = os.path.join(base_path,"fma_metadata")

# In[18]:

metadata_file = os.path.join(train_path,"metadata.json")


categories_labels_path = os.path.join(train_path,"labels.json")


In [62]:


def __load_json__(path):
    with open(path, 'r') as f:
        tmp = json.loads(f.read())

    return tmp




In [63]:

def create_dir(path):
    # checking if the directory demo_folder2 
    # exist or not.
    if not os.path.isdir(path):

        # if the demo_folder2 directory is 
        # not present then create it.
        os.makedirs(path)
    return True



In [64]:
create_dir(train_path)

True

## Load genres file. Contains relationships beetwen genres

In [65]:
genres_df = pd.read_csv(os.path.join(metadata_path_fma,'genres.csv'))


In [66]:
genres_df

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


In [67]:
genres_df[genres_df['genre_id'] == 495]

,genre_id,#tracks,parent,title,top_level
136,495,2061,15,Downtempo,15


In [68]:
# Cria um dicionário que associa o ID de cada música aos IDs de seus gêneros musicais
tracks_df = pd.read_csv(os.path.join(metadata_path_fma,'tracks_valid.csv'))

In [69]:
tracks_df = tracks_df.sample(frac=args.sample_size)

In [70]:
tracks_df.sample(20)

,track_id,track_title,valid_genre
3463,6638,Erosion of Civil Liberties/Untitled,['97']
65719,101920,Dolphin Island,"['3', '85', '103']"
18307,29768,Level Completed,"['15', '297']"
25475,40023,C'mon,['12']
37964,58316,Black One,"['2', '12', '38']"
38677,59172,Ethio Song,"['3', '33']"
26356,41164,Aïbug,['236']
55862,84481,Variation 5,"['38', '468', '514']"
76430,117465,Change,['66']
66186,103885,Transcendental Ego (ft. Vladislav Žemba) .mp3,"['1', '2', '38']"


In [71]:
tracks_df.valid_genre.values

array(["['3', '27', '33', '58', '76', '103']", "['79', '502']",
       "['11', '15']", ..., "['18']", "['17', '18', '38']",
       "['5', '18', '107']"], dtype=object)

In [72]:
tracks_df.track_title

66010                                    A Yellow Moth
65572                                             Nena
68219    You do not touch the earth, you are the earth
57446                                              One
32824                                      Beach Party
                             ...                      
39361                   The Snow That Never Falls Down
4163                                          Track 15
65461                                            Slide
93928                              Pirates libertaires
68099                                     Waking Stars
Name: track_title, Length: 104186, dtype: object

In [73]:
## Get complete genre structure
def get_all_structure(estrutura,df_genres):
    ## Get structure from df_genres
    def get_all_structure_from_df(estrutura,df_genres,structure=[]):
        if estrutura == 0:
            return structure
        else:
            structure.append(int(estrutura))
            get_all_structure_from_df(df_genres[df_genres["genre_id"]==int(estrutura)].parent.values[0],df_genres,structure)
            return structure
    
    return get_all_structure_from_df(estrutura,df_genres,structure=[])
    

In [74]:
# tracks_df['valid_genre'] = tracks_df.track_genres.apply(lambda x: x.strip('][').split(', ') if x != '[]' else None)
tracks_df['valid_genre'] = tracks_df.valid_genre.apply(lambda x: ast.literal_eval(x))

In [75]:
tracks_df['last_genre_id'] = tracks_df.valid_genre.apply(lambda x:x[-1] if x != None else None)

In [76]:
tracks_df.sample(20)

,track_id,track_title,valid_genre,last_genre_id
97852,146010,Skärhamn,"[15, 66]",66
2231,3950,Click Click Bang Bang,"[12, 85]",85
58079,87333,Gabe and Yas - The Ting Song,"[15, 38, 811]",811
94203,140839,Likelihood Of Confusion,[58],58
27343,42380,Indigenous Disguised,"[3, 27, 33, 38, 49, 94, 103]",103
93683,140085,Camp Lily Dale,"[33, 47]",47
61379,94175,Fons will fix it,"[5, 18]",18
56251,84945,spineapple,"[32, 41]",41
44974,67323,Honours,"[27, 76]",76
98051,146313,Cat's Eye,"[18, 456, 1235]",1235


In [77]:
tracks_df.dropna(inplace=True)

In [78]:
tracks_df

,track_id,track_title,valid_genre,last_genre_id
66010,103580,A Yellow Moth,"[3, 27, 33, 58, 76, 103]",103
65572,101111,Nena,"[79, 502]",502
68219,106704,"You do not touch the earth, you are the earth","[11, 15]",15
57446,86412,One,"[4, 97]",97
32824,51008,Beach Party,[2],2
...,...,...,...,...
39361,59987,The Snow That Never Falls Down,"[4, 15, 26, 42, 456]",456
4163,8370,Track 15,[22],22
65461,100857,Slide,[18],18
93928,140400,Pirates libertaires,"[17, 18, 38]",38


In [79]:
tracks_df['full_genre_id'] = tracks_df.last_genre_id.progress_apply(lambda x: get_all_structure(x,genres_df)[::-1])

  0%|          | 0/104185 [00:00<?, ?it/s]

In [80]:
tracks_df.full_genre_id.value_counts()

[38]             6508
[1235]           6043
[10, 76]         4126
[1235, 107]      3988
[17, 103]        3482
                 ... 
[20, 65, 189]       4
[9, 651, 493]       4
[2, 86, 173]        4
[20, 7]             2
[5, 444]            2
Name: full_genre_id, Length: 159, dtype: int64

In [81]:
tracks_df.columns

Index(['track_id', 'track_title', 'valid_genre', 'last_genre_id',
       'full_genre_id'],
      dtype='object')

In [82]:
tracks_df = tracks_df[['track_id','full_genre_id']]

In [83]:
tracks_df.full_genre_id.values

array([list([17, 103]), list([2, 46, 502]), list([15]), ...,
       list([1235, 18]), list([38]), list([1235, 107])], dtype=object)

In [84]:
tracks_df.full_genre_id.info

<bound method Series.info of 66010       [17, 103]
65572    [2, 46, 502]
68219            [15]
57446         [4, 97]
32824             [2]
             ...     
39361       [38, 456]
4163         [38, 22]
65461      [1235, 18]
93928            [38]
68099     [1235, 107]
Name: full_genre_id, Length: 104185, dtype: object>

In [85]:
labels_size = tracks_df.full_genre_id.apply(lambda x: len(x))

In [86]:
labels_size = labels_size.max()

In [87]:
labels_size

5

In [88]:
import pandas as pd
import os

### Parse of label to structure

In [89]:
### Function for parse label to sctructure of hierarhical scheme

def parse_label(label,label_size=5):
    # label = label.split('-')
    # preencher com 0 no caso de haver menos de 5 níveis
    labels = np.zeros(label_size,dtype=int)
    for i, label in enumerate(label):
        if i == 5:
            break
        # Aqui você pode fazer a conversão do label em um índice inteiro usando um dicionário ou outro método
        # Neste exemplo, estou apenas usando a posição da label na lista como índice
        labels[i] = label
    return labels

In [90]:
parsed_labels = tracks_df.full_genre_id.apply(lambda x: parse_label(x))

In [91]:
tracks_df['full_genre_id']

66010       [17, 103]
65572    [2, 46, 502]
68219            [15]
57446         [4, 97]
32824             [2]
             ...     
39361       [38, 456]
4163         [38, 22]
65461      [1235, 18]
93928            [38]
68099     [1235, 107]
Name: full_genre_id, Length: 104185, dtype: object

In [92]:
def convert_label_to_string(x,level=2):
    return '-'.join([str(value) for value in x[:level]])

In [93]:
tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))
tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))
tracks_df['labels_3'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=3))
tracks_df['labels_4'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=4))
tracks_df['labels_5'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=5))

  0%|          | 0/104185 [00:00<?, ?it/s]

/tmp/ipykernel_211846/310633048.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_1'] = parsed_labels.progress_apply(lambda x: str(x[:1][0]))


  0%|          | 0/104185 [00:00<?, ?it/s]

/tmp/ipykernel_211846/310633048.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df['labels_2'] = parsed_labels.progress_apply(lambda x: convert_label_to_string(x,level=2))


  0%|          | 0/104185 [00:00<?, ?it/s]

  0%|          | 0/104185 [00:00<?, ?it/s]

  0%|          | 0/104185 [00:00<?, ?it/s]

In [94]:
tracks_df.labels_1.value_counts()

38      22066
12      21710
15      18492
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [95]:
tracks_df['labels_1'].value_counts()

38      22066
12      21710
15      18492
1235    11214
17       6900
10       6514
21       6453
2        3331
5        2456
20       1183
4        1147
9        1023
8         620
13        466
14        430
3         180
Name: labels_1, dtype: int64

In [96]:
# tracks_df = tracks_df[tracks_df['labels_1'].isin(['3','14'])]

In [97]:
tracks_df.to_csv(os.path.join(train_path,"tracks.csv"),index=False)

In [98]:
tracks_df = pd.read_csv(os.path.join(train_path,"tracks.csv"))

In [99]:
categories_df = pd.DataFrame({'level5':tracks_df.labels_5.unique()})

In [100]:
categories_df['level1'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:1]))
categories_df['level2'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:2]))
categories_df['level3'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:3]))
categories_df['level4'] = categories_df.level5.progress_apply(lambda x: '-'.join(x.split('-')[:4]))

  0%|          | 0/159 [00:00<?, ?it/s]

  0%|          | 0/159 [00:00<?, ?it/s]

  0%|          | 0/159 [00:00<?, ?it/s]

  0%|          | 0/159 [00:00<?, ?it/s]

In [101]:
categories_df

,level5,level1,level2,level3,level4
0,17-103-0-0-0,17,17-103,17-103-0,17-103-0-0
1,2-46-502-0-0,2,2-46,2-46-502,2-46-502-0
2,15-0-0-0-0,15,15-0,15-0-0,15-0-0-0
3,4-97-0-0-0,4,4-97,4-97-0,4-97-0-0
4,2-0-0-0-0,2,2-0,2-0-0,2-0-0-0
...,...,...,...,...,...
154,2-46-808-0-0,2,2-46,2-46-808,2-46-808-0
155,2-86-174-0-0,2,2-86,2-86-174,2-86-174-0
156,20-7-0-0-0,20,20-7,20-7-0,20-7-0-0
157,5-444-0-0-0,5,5-444,5-444-0,5-444-0-0


In [102]:
def get_labels_name(x,genres_df):
    levels = 5
    full_name = []
    last_level = 0
    genre_root = ""
    for genre in x.split('-'):
        genre_df = genres_df[genres_df['genre_id'] == int(genre)]
        if genre_df.empty:
            genre_name = genre_root 
        else:
            genre_name = genre_df.title.values.tolist()[0]
            genre_root = genre_name
        
        full_name.append(genre_name)
    full_name = '>'.join(full_name)
        
    return full_name
    # return genres_df[genres_df['genre_id'] == int(x)].title.values.tolist()[0]

In [103]:
categories_df['level5_name'] = categories_df.level5.apply(lambda x: get_labels_name(x,genres_df))

In [104]:
categories_df

,level5,level1,level2,level3,level4,level5_name
0,17-103-0-0-0,17,17-103,17-103-0,17-103-0-0,Folk>Singer-Songwriter>Singer-Songwriter>Singe...
1,2-46-502-0-0,2,2-46,2-46-502,2-46-502-0,International>Latin America>Cumbia>Cumbia>Cumbia
2,15-0-0-0-0,15,15-0,15-0-0,15-0-0-0,Electronic>Electronic>Electronic>Electronic>El...
3,4-97-0-0-0,4,4-97,4-97-0,4-97-0-0,Jazz>Jazz: Out>Jazz: Out>Jazz: Out>Jazz: Out
4,2-0-0-0-0,2,2-0,2-0-0,2-0-0-0,International>International>International>Inte...
...,...,...,...,...,...,...
154,2-46-808-0-0,2,2-46,2-46-808,2-46-808-0,International>Latin America>Salsa>Salsa>Salsa
155,2-86-174-0-0,2,2-86,2-86-174,2-86-174-0,International>Indian>South Indian Traditional>...
156,20-7-0-0-0,20,20-7,20-7-0,20-7-0-0,Spoken>Comedy>Comedy>Comedy>Comedy
157,5-444-0-0-0,5,5-444,5-444-0,5-444-0-0,Classical>Symphony>Symphony>Symphony>Symphony


In [105]:
def __create_labels__(categories_df):
    data = {
        "label1": {},
        "label2": {},
        "label3": {},
        "label4": {},
        "label5": {},
        "label1_inverse": [],
        "label2_inverse": [],
        "label3_inverse": [],
        "label4_inverse": [],
        "label5_inverse": [],
        "label1_name": {},
        "label2_name": {},
        "label3_name": {},
        "label4_name": {},
        "label5_name": {},
    }

    idx = 0
    
    for id_x, cat in enumerate(set(categories_df.level1.values.tolist())):
        data['label1'][cat] = idx
        data['label1_inverse'].append(cat)
        data['label1_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level2.values.tolist())):
        data['label2'][cat] = idx
        data['label2_inverse'].append(cat)
        data['label2_count'] = idx + 1
        idx+=1
        
    for id_x, cat in enumerate(set(categories_df.level3.values.tolist())):
        data['label3'][cat] = idx
        data['label3_inverse'].append(cat)
        data['label3_count'] = idx + 1
        idx+=1

    for id_x, cat in enumerate(set(categories_df.level4.values.tolist())):
        data['label4'][cat] = idx
        data['label4_inverse'].append(cat)
        data['label4_count'] = idx + 1
        idx+=1
        
    for idx, cat in enumerate(set(categories_df.level5.values.tolist())):
        data['label5'][cat] = idx
        data['label5_inverse'].append(cat)
        data['label5_count'] = idx + 1
        idx+=1
        
    for cat5,cat1,cat2,cat3,cat4,name5 in categories_df.values:
        
        name1 = '>'.join(name5.split('>')[:1])
        name2 = '>'.join(name5.split('>')[:2])
        name3 = '>'.join(name5.split('>')[:3])
        name4 = '>'.join(name5.split('>')[:4])
        
        
        data['label1_name'][cat1] = name1
        data['label2_name'][cat2] = name2
        data['label3_name'][cat3] = name3
        data['label4_name'][cat4] = name4
        data['label5_name'][cat5] = name5
        
    return data

In [106]:
with open(categories_labels_path, 'w+') as f:
    f.write(json.dumps(__create_labels__(categories_df)))

In [107]:
labels  = __create_labels__(categories_df)

In [108]:
labels['label4']

{'2-130-0-0': 293,
 '12-45-53-90': 294,
 '2-92-0-0': 295,
 '2-171-0-0': 296,
 '2-86-0-0': 297,
 '2-86-174-0': 298,
 '15-296-0-0': 299,
 '5-441-0-0': 300,
 '12-27-0-0': 301,
 '15-337-0-0': 302,
 '2-130-524-0': 303,
 '21-83-0-0': 304,
 '20-65-166-0': 305,
 '12-0-0-0': 306,
 '12-31-101-0': 307,
 '2-102-0-0': 308,
 '2-46-808-0': 309,
 '20-428-0-0': 310,
 '38-30-0-0': 311,
 '3-567-0-0': 312,
 '2-117-0-0': 313,
 '5-187-0-0': 314,
 '15-468-491-0': 315,
 '1235-107-0-0': 316,
 '15-182-400-0': 317,
 '20-65-0-0': 318,
 '38-0-0-0': 319,
 '38-247-0-0': 320,
 '12-314-0-0': 321,
 '15-236-0-0': 322,
 '20-65-189-0': 323,
 '17-103-0-0': 324,
 '17-0-0-0': 325,
 '38-250-0-0': 326,
 '38-6-360-0': 327,
 '2-130-1156-0': 328,
 '12-25-71-0': 329,
 '2-118-0-0': 330,
 '17-33-0-0': 331,
 '12-25-109-0': 332,
 '4-37-0-0': 333,
 '13-0-0-0': 334,
 '21-100-0-0': 335,
 '17-94-0-0': 336,
 '2-46-502-0': 337,
 '9-0-0-0': 338,
 '20-188-0-0': 339,
 '14-11-0-0': 340,
 '2-102-1032-0': 341,
 '20-374-0-0': 342,
 '12-25-89-0': 3